In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/

In [3]:
import pandas as pd

In [4]:
dfr=pd.read_excel('bizbuysell.single.large.ds.xlsx')

In [5]:
dfr.head()

,Unnamed: 0,id,src,url,title,desc,local,region,location,title_loc,...,price,franchise,lease,auction,rent,location_2,nationwide,multiple_locations,relocatable,proc_loc
0,0,2067319,details,https://www.bizbuysell.com/Business-Real-Estat...,Popular Marina in Central New Jersey in Middle...,It is fully owned by a man and his wife since ...,Middlesex,New Jersey,"Middlesex County, NJ","Central New Jersey in Middlesex County, New Je...",...,2500000,False,False,False,False,"Middlesex County, NJ",False,False,False,"middlesex county, nj"
1,1,1990890,details,https://www.bizbuysell.com/Business-Real-Estat...,"High Exposure in Mays Landing, New Jersey - Bi...",AMAZING OPPORTUNITY! Priced to sell and ready ...,Mays Landing,New Jersey,"Mays Landing, NJ (Atlantic County)","Mays Landing, New Jersey",...,225000,False,False,False,False,"Mays Landing, NJ (Atlantic County)",False,False,False,"mays landing, nj (atlantic county)"
2,3,2039720,details,https://www.bizbuysell.com/Business-Real-Estat...,"Duplex, Short term or Long term in Pinellas Co...",Rare Largo/Seminole area duplex centrally loca...,Pinellas,Florida,"Pinellas County, FL","Pinellas County, Florida",...,595000,False,False,False,False,"Pinellas County, FL",False,False,False,"pinellas county, fl"
3,4,1576680,details,https://www.bizbuysell.com/Business-Real-Estat...,"Major Price Reduction Rest/Tavern/ 13,000sq. f...",Price reduced to $850. 000 MAKE AN OFFER!!!!!!...,Baltimore,Maryland,"Baltimore, MD (Baltimore City County)","Baltimore, Maryland",...,1050000,False,False,False,False,"Baltimore, MD (Baltimore City County)",False,False,False,"baltimore, md (baltimore city county)"
4,5,2087638,details,https://www.bizbuysell.com/Business-Real-Estat...,"Business & Real Estate in Casco, Michigan - Bi...",• The Business and the Real Estate are For Sal...,Casco,Michigan,"Casco, MI (Saint Clair County)","Casco, Michigan",...,699900,False,False,False,False,"Casco, MI (Saint Clair County)",False,False,False,"casco, mi (saint clair county)"


In [29]:
#df=dfr[:5000]
df=dfr

In [30]:
#df.shape

In [31]:
#df

In [32]:
import numpy as np

In [33]:
df['label_rand'] = np.random.randint(0,2, size=len(df))

In [34]:
df['label_median']=df['price'].apply(lambda p: 1 if p > df['price'].mean() else 0)

In [35]:
df['label_q75']=df['price'].apply(lambda p: 1 if p > df['price'].quantile(.75) else 0)

In [36]:
df=df[['label_rand','label_median','label_q75','desc']]

In [37]:
df

,label_rand,label_median,label_q75,desc
0,1,1,1,It is fully owned by a man and his wife since ...
1,0,0,0,AMAZING OPPORTUNITY! Priced to sell and ready ...
2,1,0,0,Rare Largo/Seminole area duplex centrally loca...
3,0,1,1,Price reduced to $850. 000 MAKE AN OFFER!!!!!!...
4,1,0,0,• The Business and the Real Estate are For Sal...
...,...,...,...,...
37768,1,0,0,"166,446 SF Hotel For Sale"
37769,1,0,0,We're changing the pizza industry by respectin...
37770,0,0,0,"222,631 SF Distribution For Sale"
37771,1,1,1,"Great ROI, consistent recurring revenue, and l..."


In [38]:
df=df.dropna()

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37771 entries, 0 to 37772
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label_rand    37771 non-null  int64 
 1   label_median  37771 non-null  int64 
 2   label_q75     37771 non-null  int64 
 3   desc          37771 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.4+ MB


In [52]:
X = df.desc
y = df.label_q75
print(X.shape)
print(y.shape)

(37771,)
(37771,)


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(28328,)
(9443,)
(28328,)
(9443,)


In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
# remove English stop words
# include 1-grams and 2-grams
# ignore terms that appear in more than 50% of the documents
# only keep terms that appear in at least 20 documents
vect = CountVectorizer(stop_words='english',ngram_range=(1, 2),max_df=0.5,min_df=20)

In [56]:
X_train_dtm = vect.fit_transform(X_train)

In [57]:
X_train_dtm

<28328x19667 sparse matrix of type '<class 'numpy.int64'>'
	with 2532727 stored elements in Compressed Sparse Row format>

In [58]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<9443x19667 sparse matrix of type '<class 'numpy.int64'>'
	with 842436 stored elements in Compressed Sparse Row format>

In [59]:
from sklearn.naive_bayes import MultinomialNB

In [60]:
nb = MultinomialNB()

In [61]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 17.8 ms, sys: 6.08 ms, total: 23.9 ms
Wall time: 22 ms


MultinomialNB()

In [62]:
y_pred_class = nb.predict(X_test_dtm)

In [63]:
words = np.array(vect.get_feature_names_out())

x = np.eye(X_test_dtm.shape[1])
probs = nb.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

good_words = words[ind[:30]]
bad_words = words[ind[-30:]]

good_prob = probs[ind[:30]]
bad_prob = probs[ind[-30:]]

print()
print("High price words\t     P(High Price | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

print()
print("Low price words\t     P(High Price | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))


High price words	     P(High Price | word)
       price million 0.96
           price 950 0.96
    knowledge highly 0.95
                 pip 0.95
               slips 0.94
    estate knowledge 0.92
        room revenue 0.92
  highly diversified 0.91
 highly advantageous 0.91
  building materials 0.91
     playbook caters 0.91
        caters multi 0.91
transparency education 0.91
 education resulting 0.91
 focused scalability 0.91
      component time 0.91
     fleet component 0.91
scalability consolidation 0.91
   tested operations 0.91
 operations playbook 0.91
         major fleet 0.91
        retail major 0.91
created transparency 0.91
         ebitda 2022 0.91
      multiple trade 0.90
           price 000 0.90
   focused collision 0.89
essential technology 0.89
      resulting high 0.89
         data driven 0.89

Low price words	     P(High Price | word)
            pool man 0.01
    millionaire pool 0.01
  compete successful 0.01
successful millionaire 0.01
         route start